In [1]:
import os 


In [2]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\MyDLProject\\Kidney_DL_Project'

In [5]:
### entity 

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
### config manager 